In [1]:
import os
import math
from datetime import datetime
import sys
import time
import numpy as np

import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras.layers as layers
import random

In [26]:
%load_ext tensorboard
!rm -rf ./logs/ 

input_shape = (2, 0)
output_shape = 2

model = tf.keras.Sequential([
        layers.Input(shape=input_shape),
        layers.Dense(2, use_bias=False),
        layers.ReLU(),
        layers.Dense(output_shape, use_bias=False),
        layers.ReLU()
    ])

model.summary()

loss = tf.keras.losses.mean_squared_error
optimizer = tf.keras.optimizers.Adam()

# tensorboard writer
current_time = datetime.now().strftime("%Y%m%d-%H%M%S")
graph_logdir = 'logs/func/%s' % current_time
graph_writer = tf.summary.create_file_writer(graph_logdir)

@tf.function
def graph_trace_function(x, y):
    logits = model(x, training=True)
    loss_value = loss(y, logits)
    tf.gradients(loss_value, x)
    return loss_value

tf.summary.trace_on(graph=True, profiler=True)
loss_value = graph_trace_function(tf.zeros(input_shape), tf.zeros(output_shape))

with graph_writer.as_default():
    tf.summary.trace_export(
        name="my_func_trace",
        step=0,
        profiler_outdir=graph_logdir)

%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 2, 2)              0         
_________________________________________________________________
re_lu_26 (ReLU)              (None, 2, 2)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 2, 2)              4         
_________________________________________________________________
re_lu_27 (ReLU)              (None, 2, 2)              0         
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________


In [27]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6009 (pid 41793), started 0:18:27 ago. (Use '!kill 41793' to kill it.)